# CS 436 CS5310 - Computer Vision - Assignment 4

*__Submission Instructions:__*
- Rename this notebook to `PA4_rollnumber.ipynb` before submission on LMS.
- Code for all the tasks must be written in this notebook (you do not need to submit any other files).
- The output of all cells must be present in the version of the notebook you submit.
- The university honor code should be maintained. Any violation, if found, will result in disciplinary action. 

In [0]:
#Import all the required libraries

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
import glob
from keras.models import load_model
from keras.applications import vgg16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls "/content/gdrive/My Drive/Colab Notebooks"
!unzip -qq "/content/gdrive/My Drive/Colab Notebooks/test-multiple_fruits.zip"
!unzip -qq "/content/gdrive/My Drive/Colab Notebooks/fruits-trainValidate.zip"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
fruits-trainValidate.zip  PA4_DeepLearning.ipynb  test-multiple_fruits.zip


## Overview

In this assignment you will be exploring a few important concepts used in the deep learning projects:
- Training image classification algorithms using Deep Learning
- Dataset Analyses 
- Testing deep learning classifier with the test data
- Fine-tuning / Transfer Learning

We will be using a customized datasets, the links to download the data are provided to you. You will also be working with pretrained models, which could be downloaded from keras applications. You are **highly** encouraged to explore the images in dataset and model architectures in order to get the most out of this assignment. 

**_Dataset:_**
- D1- Test Data for evaluating the pretrained model (VGG-16) can be found in the folder "test-multiple_fruits" [here](https://drive.google.com/drive/folders/1ViePNUqS3LmPkaW6vJ1cRb4YGyIkWpqz?usp=sharing)
- D2- Data to be used for fine-tuning VGG-16 for 75 classes of fruits could be downloaded from the same link in the folder "fruits-trainValidate"


**_Pretrained Models:_** 
Can be found [here](https://keras.io/applications/#applications)


## Task 1: Data Preparation

Evaluate the performance of a pretrained network (VGG-16) for the test-multiple_fruits data downloaded by predicting labels of each image. You will:
- Download the VGG16 model and compile it with pretrained weights from imagenet.
- Obtain predictions for the test-multiple_fruits D1 dataset
- Print the predictions for all the test image from D1

In [6]:
def pre_trained_test():
  model = VGG16()
  for fileName in glob.glob('test-multiple_fruits/*.jpg'):
    image = load_img(fileName, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat = model.predict(image)
    label = decode_predictions(yhat)
    label = label[0][0]
    print('%s (%.2f%%)' % (label[1], label[2]*100))
  
pre_trained_test()


corn (31.64%)
strawberry (25.86%)
custard_apple (84.63%)
rotisserie (62.49%)
lemon (28.92%)
barrow (11.06%)
hermit_crab (20.62%)
cucumber (15.12%)
buckeye (61.51%)
abacus (38.20%)
guacamole (62.88%)
hip (67.51%)
pomegranate (17.86%)
strawberry (61.05%)
buckeye (6.36%)
Granny_Smith (68.36%)
orange (87.63%)
tray (38.76%)
Granny_Smith (70.41%)
orange (59.01%)
pot (29.19%)
fig (28.74%)
hip (32.14%)
pomegranate (35.17%)
pineapple (22.97%)
pomegranate (65.32%)
hotdog (49.03%)
pomegranate (24.42%)
strawberry (30.33%)
mixing_bowl (31.72%)
fig (82.88%)
strawberry (93.30%)
strawberry (41.33%)
hip (23.56%)
Granny_Smith (97.14%)
teddy (13.77%)
bucket (21.57%)
strawberry (74.01%)
grocery_store (29.84%)
daisy (18.28%)
greenhouse (29.44%)
butternut_squash (10.44%)
pot (47.05%)
tray (28.33%)
honeycomb (53.67%)


## Batch Generator for Task 2
You could either use this batch image generator or could write your own batch generator if required for fine-tunning

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory('fruits-360/Training/', target_size = (100,100), batch_size = 32, class_mode = 'categorical')
validation_generator = validation_datagen.flow_from_directory('fruits-360/Validation/', target_size = (100,100), batch_size = 32, class_mode = 'categorical')

Found 37836 images belonging to 75 classes.
Found 12709 images belonging to 75 classes.


## Task 2: Transfer Learning

Next you will employ Transfer Learning and finetune the pretrained vgg-16 model you used in Task1 to better fit the fine-tune dataset D2 for 75 classes of fruits (details available in the readme file of dataset folder). You will:

- Change the number of nodes in the last FC layer according to the number of classes i.e. 5 
- Freeze everything except the FC layers and train it using the train split of D2 (using appropriate hyperparameters), validating the network for validation split of data.
- Train (Finetune) the dataset with training split and validate it using validation split
- Plot loss/accuracy vs epochs curves for your simulation

*You can use scikit-learn's `metrics.confusion_matrix` function. Consult the relevant documentation.* 


In [12]:
    
from keras import models
from keras import layers
from keras import optimizers

vgg_imagenet = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(100,100,3))
for lyr in vgg_imagenet.layers:
  lyr.training = False

 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_imagenet)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(75, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

# add new FC layers here

# print summary and compile


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              4719616   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 75)                76875     
Total params: 19,511,179
Trainable params: 19,511,179
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      training_generator,
      steps_per_epoch=50 ,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

Epoch 1/5
49/50 [============================>.] - ETA: 13s - loss: 4.5275 - acc: 0.0198

KeyboardInterrupt: ignored

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [0]:
#Loss accuracy curves using matplot lib

## Task 3: Network Evaluation

Next you will test your finetuned model by plotting a confusion matrix between classes predicted. You will:

- Load the saved finetuned network
- Test your model for images in validation folder of D2
- Construct a multiclass confusion matrix (for any 10 classes) for actual and predicted class of each image and visualize the confmatrix as a heatmap

*You can use scikit-learn's `metrics.confusion_matrix` function. Consult the relevant documentation.* 

## Task 4: Prediction

Next you will test your finetuned model for test-multiple_fruits images and compare the result with the results of Task 1. You will:

- Predict labels for images in test-multiple_fruits D1 folder using finetuned network
- Compare your result qualitativly and quatitatively (by visualizing some of the comparing images with their respective labels)
- Analyse and discuss the improvement of results (if found any)


In [0]:
import os
def makeIT():
  the_list =[]
  for filename in os.listdir("fruits-360/Training"):
    the_list.append(filename)
  return the_list


the_list = makeIT()

def printIT(the_list):
  for image in glob.glob('test-multiple_fruits/*.jpg'):
    img = load_img(image, target_size=(100, 100))
    arr = img_to_array(img)
    array = arr.reshape((1, 100, 100, 3)) 
    pred = new_model.predict(array)
    pred = pred.reshape(-1)
    num_elem = len(image.split('/'))[1].split('_')
    print("-------------------")
    print("Image: ",image)
    for i in range(num_elem):
      index = np.argmax(pred)
      print(the_list[index])
      pred[index]=-1
          
printIT(the_list)